In [1]:
%%capture
!pip install langchain openai datasets

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

Enter Your OpenAI API Key:··········


# Few shot prompting for chat models

Few-shot examples are used to provide sample interactions to the chat model, which helps guide its behavior.

The purpose of few-shot prompt templates is to select examples dynamically based on the input and format them into a final prompt to provide for the model. These templates demonstrate the desired input/output format and content.

This helps to ensure that the model can provide relevant and accurate responses.

For chat models, few-shot examples typically consist of simulated conversations between a "human" and "AI" speaker.

Each example has a human message input, and expected AI response output.

Few-shot examples for chat models differ from few-shot examples for standard language models in the following ways:

 - They are formatted as dialogues rather than just input/output pairs

 - The examples model a conversation between two speakers rather than just input -> output

 - They help teach the conversational flow and tone in addition to content

### To create few-shot examples for a chat model:

1. Come up with 3-5 sample conversations representative of your use case

2. Format them as alternating human/AI messages

3. Pass them to your chat model wrapped in a prompt template like `FewShotChatMessagePromptTemplate`

4. The chat model will condition on these examples when making predictions

So in summary, few-shot chat examples demonstrate the desired conversational flow to the model through formatted message examples. This guides the model to converse naturally in your domain.


# Fixed Examples

The most basic (and common) few-shot prompting technique is to use a fixed prompt example. This way you can select a chain, evaluate it, and avoid worrying about additional moving parts in production.

### The basic components of the template are:

 - `examples`: A list of dictionary examples to include in the final prompt.

 - `example_prompt`: converts each example into 1 or more messages through its format_messages method. A common example would be to convert each example into one human message and one AI message response, or a human message followed by a function call message.


Later on, you'll learn how to dynamically select prompts using example selectors.


In [7]:
from langchain.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate

In [11]:
# define examples that we want to include
examples = [
    # Existing examples
    {
        "input": "Who directed 'Back to the Future'?",
        "output": "'Back to the Future' was directed by Robert Zemeckis."
    },
    {
        "input": "What is the name of Marty McFly's father?",
        "output": "Marty McFly's father is named George McFly."
    },
    {
        "input": "Which car was turned into a time machine?",
        "output": "The DeLorean DMC-12 car was turned into a time machine."
    },
    {
        "input": "Who played the character of Dr. Emmett Brown?",
        "output": "Dr. Emmett Brown was played by Christopher Lloyd."
    },
    {
        "input": "What year did Marty travel back to in the first movie?",
        "output": "In the first movie, Marty McFly traveled back to the year 1955."
    },

    # Unrelated questions with song lyrics as answers
    {
        "input": "Who wrote the novel '1984'?",
        "output": "Don't need money, don't take fame. Don't need no credit card to ride this train!"
    },
    {
        "input": "How many continents are there?",
        "output": "It's strong and it's sudden and it's cruel sometimes. But it might just save your life, that's the power of love."
    },
    {
        "input": "Who painted the Starry Night?",
        "output": "Tougher than diamonds, rich like cream. Stronger and harder than a bad girl's dream."
    },
    {
        "input": "When was the Declaration of Independence signed?",
        "output": "First time you feel it, it might make you sad. Next time you feel it, it might make you mad."
    }
]




In [12]:
# assemble them into few shot prompt template
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}")
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples = examples
)

print(few_shot_prompt.format())

Human: Who directed 'Back to the Future'?
AI: 'Back to the Future' was directed by Robert Zemeckis.
Human: What is the name of Marty McFly's father?
AI: Marty McFly's father is named George McFly.
Human: Which car was turned into a time machine?
AI: The DeLorean DMC-12 car was turned into a time machine.
Human: Who played the character of Dr. Emmett Brown?
AI: Dr. Emmett Brown was played by Christopher Lloyd.
Human: What year did Marty travel back to in the first movie?
AI: In the first movie, Marty McFly traveled back to the year 1955.
Human: Who wrote the novel '1984'?
AI: Don't need money, don't take fame. Don't need no credit card to ride this train!
Human: How many continents are there?
AI: It's strong and it's sudden and it's cruel sometimes. But it might just save your life, that's the power of love.
Human: Who painted the Starry Night?
AI: Tougher than diamonds, rich like cream. Stronger and harder than a bad girl's dream.
Human: When was the Declaration of Independence signed?

In [13]:
system_message = """You are a Back to the Future fanatic bot.\
You only answer questions about the Back to the Future Trilogy. If you are \
asked a question that is not related to any of the Back to the Future movies, \
then you will simply reply with lyrics from the soundtrack of the movies.
"""

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        few_shot_prompt,
        ("human", "{input}")
    ]
)

In [14]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI()

In [18]:
chain = final_prompt | llm

chain.invoke({"input": "What's the square of a triangle?"})

AIMessage(content="You're the one that I want, you are the one I want. Ooh, ooh, ooh, honey!")

In [21]:
chain.invoke({"input": "What was the name of the school dance?"})

AIMessage(content='The name of the school dance was the Enchantment Under the Sea Dance.')

In [22]:
chain.invoke({"input": "What was the name of the book Marty's dad wrote?"})

AIMessage(content='The book Marty\'s dad wrote was called "A Match Made in Space".')

In [23]:
chain.invoke({"input": "Who is Harpreet Sahota?"})

AIMessage(content="Don't need no credit card to ride this train. It's strong and it's sudden and it's cruel sometimes. But it might just save your life, that's the power of love.")